In [17]:
val sampleData = Seq(
    (1,"Pedro"),
    (2,"Sergi"),
    (3,"Aleix"),
    (4,"David")
)

import spark.implicits._
val df = sampleData.toDF()

spark.conf.set("spark.databricks.delta.properties.defaults.enableChangeDataFeed", "true")
df.coalesce(1).write.mode("overwrite").format("delta").saveAsTable("data_engineers_cdc")

23/11/26 15:45:31 INFO DeltaLog: No delta log found for the Delta table at file:/home/jovyan/spark-warehouse/data_engineers_cdc/_delta_log
23/11/26 15:45:31 INFO InitialSnapshot: [tableId=10b3a069-3f26-4f3f-9600-8d3e43429e3a] Created snapshot InitialSnapshot(path=file:/home/jovyan/spark-warehouse/data_engineers_cdc/_delta_log, version=-1, metadata=Metadata(e378c782-fcec-49d0-a30d-6e6a65fe7417,null,null,Format(parquet,Map()),null,List(),Map(delta.enableChangeDataFeed -> true),Some(1701013531856)), logSegment=LogSegment(file:/home/jovyan/spark-warehouse/data_engineers_cdc/_delta_log,-1,List(),List(),None,-1), checksumOpt=None)
23/11/26 15:45:31 INFO MapPartitionsRDD: Removing RDD 330 from persistence list
23/11/26 15:45:31 INFO BlockManager: Removing RDD 330
23/11/26 15:45:31 INFO DeltaLog: No delta log found for the Delta table at file:/home/jovyan/spark-warehouse/data_engineers_cdc/_delta_log
23/11/26 15:45:31 INFO InitialSnapshot: [tableId=e378c782-fcec-49d0-a30d-6e6a65fe7417] Created

sampleData: Seq[(Int, String)] = List((1,Pedro), (2,Sergi), (3,Aleix), (4,David))
import spark.implicits._
df: org.apache.spark.sql.DataFrame = [_1: int, _2: string]


In [18]:
import io.delta.tables._

val newDataDf = Seq(
    (4,"Adrian"),
    (5,"David"),
    (6,"Simon")
).toDF()

DeltaTable
.forName("data_engineers_cdc")
.as("oldData")
  .merge(
    newDataDf.as("newData"),
    "oldData._1 = newData._1")
  .whenMatched
  .updateAll
  .whenNotMatched
  .insertAll
  .execute()

23/11/26 15:45:33 INFO SparkContext: Starting job: $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128
23/11/26 15:45:33 INFO DAGScheduler: Got job 82 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128) with 50 output partitions
23/11/26 15:45:33 INFO DAGScheduler: Final stage: ResultStage 127 ($anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128)
23/11/26 15:45:33 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 126)
23/11/26 15:45:33 INFO DAGScheduler: Missing parents: List()
23/11/26 15:45:33 INFO DAGScheduler: Submitting ResultStage 127 (MapPartitionsRDD[377] at $anonfun$recordDeltaOperationInternal$1 at DatabricksLogging.scala:128), which has no missing parents
23/11/26 15:45:33 INFO MemoryStore: Block broadcast_122 stored as values in memory (estimated size 573.9 KiB, free 432.0 MiB)
23/11/26 15:45:33 INFO MemoryStore: Block broadcast_122_piece0 stored as bytes in memory (estimated size 132.5 KiB, free 431.9

import io.delta.tables._
newDataDf: org.apache.spark.sql.DataFrame = [_1: int, _2: string]


In [19]:
spark.read.format("delta")
  .option("readChangeFeed", "true")
  .option("startingVersion", 0)
  .table("data_engineers_cdc").show(false)

23/11/26 15:45:34 INFO BlockManager: Removing RDD 375
23/11/26 15:45:34 INFO CDCReader: startingVersion: Some(0), endingVersion: None
23/11/26 15:45:34 INFO DeltaLog: Try to find Delta last complete checkpoint before version 0
23/11/26 15:45:34 INFO DeltaLog: No checkpoint found for Delta table before version 0
23/11/26 15:45:34 INFO DeltaLog: Loading version 0.
23/11/26 15:45:34 INFO DeltaLogFileIndex: Created DeltaLogFileIndex(JSON, numFilesInSegment: 1, totalFileSize: 1239)
23/11/26 15:45:34 INFO FileSourceStrategy: Pushed Filters: 
23/11/26 15:45:34 INFO FileSourceStrategy: Post-Scan Filters: 
23/11/26 15:45:34 INFO FileSourceStrategy: Output Data Schema: struct<metaData: struct<id: string, name: string, description: string, format: struct<provider: string, options: map<string,string>>, schemaString: string ... 6 more fields>, protocol: struct<minReaderVersion: int, minWriterVersion: int, readerFeatures: array<string>, writerFeatures: array<string> ... 2 more fields>>
23/11/26 15:4

In [21]:
spark.read.format("parquet").load("spark-warehouse/data_engineers_cdc/_change_data/cdc-00000-540b06a2-4586-48e4-af82-7e48ffdd89c4.c000.snappy.parquet").show(false)

23/11/26 15:52:49 INFO InMemoryFileIndex: It took 5 ms to list leaf files for 1 paths.
23/11/26 15:52:49 INFO SparkContext: Starting job: load at <console>:55
23/11/26 15:52:49 INFO DAGScheduler: Got job 97 (load at <console>:55) with 1 output partitions
23/11/26 15:52:49 INFO DAGScheduler: Final stage: ResultStage 150 (load at <console>:55)
23/11/26 15:52:49 INFO DAGScheduler: Parents of final stage: List()
23/11/26 15:52:49 INFO DAGScheduler: Missing parents: List()
23/11/26 15:52:49 INFO DAGScheduler: Submitting ResultStage 150 (MapPartitionsRDD[448] at load at <console>:55), which has no missing parents
23/11/26 15:52:49 INFO MemoryStore: Block broadcast_147 stored as values in memory (estimated size 104.0 KiB, free 431.6 MiB)
23/11/26 15:52:49 INFO MemoryStore: Block broadcast_147_piece0 stored as bytes in memory (estimated size 37.4 KiB, free 431.6 MiB)
23/11/26 15:52:49 INFO BlockManagerInfo: Added broadcast_147_piece0 in memory on 0541434f17db:43915 (size: 37.4 KiB, free: 433.9